# Identifying (volatility-)regimes in the the EUR/USD spot exchange rate using clustering algorithms: An Oil and Gas Perspective on Parity Conditions.
Seminar in Applied Financial Economics: Applied Econometrics of FX Markets - Professor Reitz
<br>
**Josef Fella and Robert Hennings**
<br>
Christian Albrechts University of Kiel
<br>
*josef.fella@stu.uni-kiel.de and robert.hennings@stu.uni-kiel.de*
<br>
GitHub: https://github.com/RobertHennings
<br>
Kiel - 14.11.2025

## Outline
1. Research Hypothesis
<br>
1.1 Energy Commodity Price Shocks: The Pass-Through Effect and Implications for Monetary Policy
<br>
1.2 Formulated Research Hypothesis
<br>
2. Literature Review
<br>
2.1 Systematic Literature Overview: Main Approaches
<br>
3. Theoretical Framework
<br>
3.1 Impact of Inflation on Measurements: What are prices and how are they measured?
<br>
3.2 A simple model of exchange rates and commodity prices
<br>
3.3. Theoretical Framework
<br>
4. Methodology Overview
<br>
5. Model Results
<br>
5.1 Regime identification - Model comparison and selection
<br>
6. Conclusion and Discussion
<br>
6.1 Seminar Project Summary
<br>
6.2 Seminar Project Limitations
<br>
6.3 Future Research
<br>
7. Appendix
<br>
7.1 Figures and Tables
<br>
7.2 Data and Definitions
<br>

Import dependencies/packages:

In [1]:
import logging
import json
import os
import pandas as pd
import numpy as np
import yfinance as yf

Set global config settings:

**!!!!CHANGE WORKING DIRECTORY HERE!!!!**

In [ ]:
SEMINAR_PATH = r"/Users/Robert_Hennings/Uni/Master/Seminar"

In [ ]:
SEMINAR_CODE_PATH = rf"{SEMINAR_PATH}/src/seminar_code"
MODELS_PATH = rf"{SEMINAR_CODE_PATH}/models"
FIGURES_PATH = rf"{SEMINAR_PATH}/reports/figures"
TABLES_PATH = rf"{SEMINAR_PATH}/reports/tables"
DATA_PATH = rf"{SEMINAR_PATH}/data"
PRESENTATION_DATA = rf"{SEMINAR_PATH}/reports/presentation_latex_version/data"
NUM_YEARS_INTERVAL_X_AXIS = 5
RENDERER = "vscode+browser"
CAU_COLOR_SCALE = ["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"]
print(os.getcwd())
os.chdir(SEMINAR_CODE_PATH)
print(os.getcwd())

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code
/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code


In [12]:
from utils.evaluation import adf_test, \
    granger_causality_test, \
    cointegration_test, \
    test_data_for_normality, \
    extract_predicted_labels_from_metadata_df,\
    get_recoded_predicted_labels_df, \
    get_regime_counts_df, \
    get_overlapping_regimes_df, \
    get_periods_overlaying_df, \
    get_model_metadata_df

# Import custom written model evaluation scores
from utils.evaluation_metrics import compute_rcm

from data_loading.data_loader import DataLoading
from data_graphing.data_grapher import DataGraphing

# Instantiate the data loading class
data_loading_instance = DataLoading(
    credential_path=r"/Users/Robert_Hennings/Projects/SettingsPackages",
    credential_file_name=r"credentials.json"
)
# Instantiate the data graphing class
data_graphing_instance = DataGraphing()

INFO:root:Credentials loaded from file: credentials.json in path: /Users/Robert_Hennings/Projects/SettingsPackages


## 0 - Intro: The PPP puzzle and Commodity Currencies

In [6]:
file_name = r"chap_00_exchange_rates_ppp_deviations.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
exchange_rates_df = pd.read_excel(full_file_path, index_col=0)

effective_exchange_rates_df = pd.DataFrame()
effective_exchange_rates_df["GBPREAL"] = np.log(exchange_rates_df["United Kingdom"]) - np.log(exchange_rates_df["United States"])
effective_exchange_rates_df["JPYREAL"] = np.log(exchange_rates_df["Japan"]) - np.log(exchange_rates_df["United States"])
effective_exchange_rates_df["CHFREAL"] = np.log(exchange_rates_df["Switzerland"]) - np.log(exchange_rates_df["United States"])
effective_exchange_rates_df["EUROREAL"] = np.log(exchange_rates_df["Euro Area (EA)"]) - np.log(exchange_rates_df["United States"])


data = effective_exchange_rates_df.copy()
variables = data.columns.tolist()
secondary_yaxis_variables = []
title = f"Monthly deviations of USD Spot Rate from PPP-values over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title = "Time"
y_axis_title = "Deviations of USD Spot Rate from PPP-values (log)"
secondary_yaxis_title = ""
color_mapping = {
    'GBPREAL': "grey",
    'JPYREAL': "black",
    'CHFREAL': "#9b0a7d",
    'EUROREAL': "darkgrey"
}
fig_deviations_from_ppp = data_graphing_instance.get_fig_deviations_ppp(
        data=data,
        variables=variables,
        secondary_y_variables=secondary_yaxis_variables,
        title="",
        secondary_y_axis_title=secondary_yaxis_title,
        color_discrete_sequence=CAU_COLOR_SCALE,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_00_deviations_of_usd_spotrates_from_ppp_values",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_deviations_from_ppp.show(renderer=RENDERER)

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/data_graphing/data_grapher.py:1177: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## 1 - Research Hypothesis

### 1.1 Energy Commodity Price Shocks: The Pass-Through Effect and implications for Monetary Policy

#### Energy Price Contributions to Inflation - USA

In [7]:
file_name = r"chap_01_us_inflation_contribution_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
data_us_df = pd.read_excel(full_file_path, index_col=0)

series_dict_mapping = {
    'Headline': 'CPIAUCSL',
    'Core CPI': 'CPILFESL',
    'Energy': 'CPIENGSL',
    'Food': 'CPIUFDSL',
}
weights = {
    'Core CPI': 1 - 0.136 - 0.072,  # Core (everything except food & energy)
    'Energy': 0.072,
    'Food': 0.136,
}
start_date = '2000-01-01'
end_date = '2024-12-31'
headline_id = 'CPIAUCSL'  

def percent_change(series):
    return 100 * (series.iloc[-1] / series.iloc[-13] - 1)

# Compute 12-month percent change (annual inflation): (P_t / P_{t-12} - 1) * 100
data_pct = data_us_df.pct_change(periods=12) * 100

# Compute component contributions: weight × component inflation
contributions = pd.DataFrame(index=data_pct.index)
for col in weights:
    contributions[col] = data_pct[col] * weights[col]

# Only these bar components
bar_components = weights.keys()
contributions = contributions[bar_components].dropna()

# Sort contributors by value for each period (row), biggest on bottom
contributions_sorted = pd.DataFrame(
    np.sort(contributions[bar_components].values, axis=1)[:, ::-1],  # sort and reverse for descending
    index=contributions.index,
    columns=[f"{i+1}" for i in range(len(bar_components))]
)

# Next, get the names for the columns for legend and color mapping
def get_sorted_labels(row):
    # Sort values for the row, get corresponding labels
    vals = row.values
    labels = [x for _, x in sorted(zip(vals, bar_components), reverse=True)]
    return labels

labels_sorted = contributions[bar_components].apply(get_sorted_labels, axis=1)

variables = list(data_us_df.columns)
energy_color = "#9b0a7d"
cpi_color = "red"  # Fixed color for the "World" category

color_mapping = {
    'Core CPI': "grey",
    'Energy': energy_color,
    'Food': "lightgrey",
    'Headline': cpi_color,
}
title = f"US CPI: Headline and component contributions over the time: {contributions.index[0].year} - {contributions.index[-1].year}"
x_axis_title = "Time"
y_axis_title = 'Contribution to annual inflation (%)'

fig_inflation_decomp_usa = data_graphing_instance.get_fig_inflation_contribution_usa(
        data=contributions,
        data_pct=data_pct,
        cpi_color=cpi_color,
        variables=list(bar_components),
        title="",
        secondary_y_variables=[],
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_02_us_cpi_inflation_decomposition",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
fig_inflation_decomp_usa.show(renderer=RENDERER)

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/data_graphing/data_grapher.py:882: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



#### Energy Price Contributions to Inflation - EU area

In [8]:
file_name = r"chap_01_eu_inflation_contribution_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
data = pd.read_excel(full_file_path, index_col=0)

weights = {
    "Food": 0.172,      # 17.2%
    "Energy": 0.098,    # 9.8%
    "Core CPI": 1 - 0.172 - 0.098,  # remainder
}
data = data.resample('M').last()  # If needed, ensure monthly end-of-period alignment

# Here, each value is ALREADY YoY % change. So simply multiply by weights for contributions:
contributions = pd.DataFrame(index=data.index)
for col in weights:
    contributions[col] = data[col] * weights[col]

bar_components = data.columns.difference(['Headline'])
contributions = contributions[bar_components].dropna()
# Sort contributors by value for each period (row), biggest on bottom
contributions_sorted = pd.DataFrame(
    np.sort(contributions[bar_components].values, axis=1)[:, ::-1],  # sort and reverse for descending
    index=contributions.index,
    columns=[f"{i+1}" for i in range(len(bar_components))]
)
labels_sorted = contributions[bar_components].apply(get_sorted_labels, axis=1)

variables = list(contributions.columns)
contributions["Headline"] = data["Headline"]

energy_color = "#9b0a7d"
cpi_color = "red"  # Fixed color for the "World" category

color_mapping = {
    'Core CPI': "grey",
    'Energy': energy_color,
    'Food': "lightgrey",
    'Headline': cpi_color,
}
title = f"EU Area CPI: Headline and component contributions over the time: {contributions.index[0].year} - {contributions.index[-1].year}"
x_axis_title = "Time"
y_axis_title = 'Contribution to annual inflation (%)'

fig_inflation_decomp_euro_area = data_graphing_instance.get_fig_inflation_contribution_euro_area(
        data=contributions,
        cpi_color=cpi_color,
        variables=variables,
        secondary_y_variables=[],
        title="",
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_02_eu_area_cpi_inflation_decomposition",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_inflation_decomp_euro_area.show(renderer=RENDERER)

/var/folders/5j/5rywkp2x7zs75k_ml53cn10w0000gn/T/ipykernel_16995/3113023776.py:10: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/data_graphing/data_grapher.py:1014: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



#### Rolling Volatility of Exchange Rates and Energy Commodity Prices

In [9]:
file_name = r"chap_01_daily_exchange_rate_oil_log_diff_vola_normalized_crisis_periods_highlighted.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
data_log_first_diff = pd.read_excel(full_file_path, index_col=0)

file_name = r"crisis_periods_dict.json"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"

with open(full_file_path, "r") as f:
    crisis_periods_dict = json.load(f)

data_log_diff_normalized = (data_log_first_diff - data_log_first_diff.min()) / (data_log_first_diff.max() - data_log_first_diff.min())
title=f"Daily normalized EUR/USD spot exchange rate, oil and gas log first differences volatility with highlighted crisis periods over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title="Time"
y_axis_title="Volatility of log first differences (normalized)"


fig_crisis_periods_highlighted = data_graphing_instance.get_fig_crisis_periods_highlighted(
    data=data_log_diff_normalized,
    crisis_periods_dict=crisis_periods_dict,
    variables=data_log_diff_normalized.columns,
    secondary_y_variables=[],
    recession_shading_color="rgba(155, 10, 125, 0.3)",
    title="",
    secondary_y_axis_title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    color_mapping_dict={
        'EUR/USD': "grey",
        'WTI Oil': "black",
        'Natural Gas': "#9b0a7d",
    },
    num_years_interval_x_axis=5,
    showlegend=False,
    save_fig=False,
    file_name="chap_02_daily_exchange_rate_oil_log_diff_vola_normalized_crisis_periods_highlighted",
    file_path=FIGURES_PATH,
    width=1200,
    height=800,
    scale=3
    )
# Show the figure
fig_crisis_periods_highlighted.show(renderer=RENDERER)

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/data_graphing/data_grapher.py:1685: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



## 5 - Model Results

### 5.1 Regime identification - Model comparison and selection

#### Model comparison and selection

In [13]:
all_models_comp_df = get_model_metadata_df(
    full_model_info_path=MODELS_PATH,
    )
all_models_comp_df[["model_type", "silhouette_score", "feature_names_in"]].sort_values(by="silhouette_score", ascending=False)

# Plot the model training results as a bar plot with the used features as index
unique_df = all_models_comp_df.drop_duplicates(subset=["model_type", "silhouette_score", "feature_names_in"]).dropna(subset=["silhouette_score"])

predicted_labels_df = extract_predicted_labels_from_metadata_df(
    metadata_df=all_models_comp_df,
)
for column in predicted_labels_df.columns:
    if predicted_labels_df[column].max() > 1 or predicted_labels_df[column].min() <0 or (predicted_labels_df[column].dropna() == 0.0).all():
        print(f"Removing column: {column}")
        predicted_labels_df = predicted_labels_df.drop(columns=[column])

keep_models = [model.replace(":", "-") for model in predicted_labels_df.columns]
keep_models = [model.replace(" ", "_") for model in keep_models]
exact_model = unique_df["model_type"] + "_" + unique_df["time_last_fitted"].str.replace(":", "-").str.replace(" ", "_")
unique_df["model_file_name"] = exact_model
unique_df = unique_df[unique_df["model_file_name"].isin(keep_models)]
title="Model comparison using the silhouette score (1 being best, 0 indicating overlapping clusters, -1 being worst) for various regime identification model configurations"

fig_model_comp_bar_plot = data_graphing_instance.get_model_comparison_bar_plot(
    data=unique_df,
    evaluation_score_col_name="silhouette_score",
    title="",
    x_axis_title="Feature names",
    y_axis_title="Silhouette score (1 best - 0 overlapping clusters - -1 worst)",
    color_mapping_dict={
        "KMeans": "black",
        "AgglomerativeClustering": "darkgrey",
        "DBSCAN": "blue",
        "MeanShift": "red",
        "MarkovRegression": "#9b0a7d",
        "GaussianMixture": "lightgrey",
        "Birch": "pink",
        "AffinityPropagation": "gray",
        "OPTICS": "olive",
        "MiniBatchKMeans": "#00677c"
    },
    save_fig=False,
    file_name="model_comparison_bar_plot",
    file_path=FIGURES_PATH,
    showlegend=True,
    textfont_size=8.5,
    width=1400,
    height=800,
    scale=3
    )
fig_model_comp_bar_plot.show(renderer=RENDERER)

INFO:root:Found 65 model info files in /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/Birch_2025-10-15_17-20-24 in format .json


INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/AffinityPropagation_2025-10-15_17-06-20 in format .json
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/GaussianMixture_2025-10-15_17-12-07 in format .json
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/DBSCAN_2025-10-15_17-03-29 in format .json
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/GaussianMixture_2025-10-15_17-17-04 in format .json
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/Birch_2025-10-15_17-17-05 in format .json
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/Birch_2025-10-15_17-04-14 in format .json
INFO:root:Loading model info from /Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/models/MeanShift_202

Removing column: Birch_2025-10-15 17:20:24
Removing column: AffinityPropagation_2025-10-15 17:06:20
Removing column: DBSCAN_2025-10-15 17:03:28
Removing column: Birch_2025-10-15 17:04:14
Removing column: MeanShift_2025-10-15 17:07:31
Removing column: Birch_2025-10-15 17:12:07
Removing column: DBSCAN_2025-10-15 17:10:33
Removing column: DBSCAN_2025-10-15 17:19:42
Removing column: MeanShift_2025-10-15 17:20:17
Removing column: DBSCAN_2025-10-15 20:46:38
Removing column: OPTICS_2025-10-15 17:10:20
Removing column: AffinityPropagation_2025-10-15 17:10:01
Removing column: MeanShift_2025-10-15 17:16:57
Removing column: AffinityPropagation_2025-10-15 20:49:20
Removing column: OPTICS_2025-10-15 17:06:29
Removing column: DBSCAN_2025-10-15 17:14:47
Removing column: AffinityPropagation_2025-10-15 17:19:16
Removing column: Birch_2025-10-15 17:07:37
Removing column: AffinityPropagation_2025-10-15 17:14:14
Removing column: MeanShift_2025-10-15 17:04:09
Removing column: OPTICS_2025-10-15 17:22:41
Rem

#### Predicted Regimes - Evolution over time with highlighted Crisis Periods

In [ ]:
# Now also map the regimes across the models correctly, in that we assume the high vola regime is encoded as 1 and the low vola regime is a 0
# KMeans is already correctly encoded
# AgglomerativeClustering needs to be encoded
# MeanShift is also correctly encoded
# MarkovRegression needs to be encoded
predicted_labels_df = get_recoded_predicted_labels_df(
    predicted_labels_df=predicted_labels_df,
    label_mapping_dict={0: 1, 1: 0},
    column_names_list=["AgglomerativeClustering", "MarkovRegression"]
    )

series_dict_mapping = {
    'USD/EUR': 'DEXUSEU',
    'WTI Oil': 'DCOILWTICO',
    "Nat Gas": "DHHNGSP",
}
start_date = central_bank_policy_rate_df.index.min().strftime('%Y-%m-%d')
end_date = central_bank_policy_rate_df.index.max().strftime('%Y-%m-%d')

data_dict, data_full_info_dict, lowest_freq = data_loading_instance.get_fred_data(
    series_dict_mapping=series_dict_mapping,
    start_date=start_date,
    end_date=end_date
    )
spot_exchange_rate_data_df = pd.concat(list(data_dict.values()), axis=1).dropna()
# Compute the EUR/USD rate
spot_exchange_rate_data_df["EUR/USD"] = 1 / spot_exchange_rate_data_df["USD/EUR"]
spot_exchange_rate_data_df = spot_exchange_rate_data_df.drop(columns=["USD/EUR"])

predicted_labels_df.index = spot_exchange_rate_data_df.index

# Now also load in the table of crisis periods and overlay these time periods as well
# to see if regime changes have been picked up
graphing_df = pd.concat(
    [spot_exchange_rate_data_df, predicted_labels_df],
    axis=1
)

file_name = r"crisis_periods_dict.json"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"

with open(full_file_path, "r") as f:
    crisis_periods_dict = json.load(f)


custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(graphing_df.columns)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(graphing_df.columns.tolist())}


fig_crisis_periods_highlighted = data_graphing_instance.get_fig_crisis_periods_highlighted(
    data=graphing_df,
    crisis_periods_dict=crisis_periods_dict,
    variables=["rolling_vola"],
    secondary_y_variables=graphing_df.columns[1:].tolist(),
    recession_shading_color="rgba(155, 10, 125, 0.3)",
    title=f"Exchange Rate and Oil Volatility with Crisis Periods highlighted over the time: {graphing_df.index[0].year} - {graphing_df.index[-1].year}",
    secondary_y_axis_title="WTI Oil & Natural Gas Volatility",
    x_axis_title="Date",
    y_axis_title="Exchange Rate Volatility",
    color_mapping_dict=color_mapping,
    num_years_interval_x_axis=5,
    showlegend=True,
    save_fig=False,
    file_name="exchange_rate_oil_raw_vola_crisis_periods_highlighted",
    file_path=FIGURES_PATH,
    width=1200,
    height=800,
    scale=3
    )
# Show the figure
fig_crisis_periods_highlighted.show(renderer="browser")

## 7 - Appendix

### 7.1 Figures and Tables

In [14]:
file_name = r"chap_07_oil_consumption_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
oil_consumption_df = pd.read_excel(full_file_path, index_col=0)

file_name = r"chap_07_oil_production_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
oil_production_df = pd.read_excel(full_file_path, index_col=0)

file_name = r"chap_07_gas_consumption_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
gas_consumption_df = pd.read_excel(full_file_path, index_col=0)

file_name = r"chap_07_gas_production_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
gas_production_df = pd.read_excel(full_file_path, index_col=0)

#### Oil: Global Production and Consumption over time

In [ ]:
variables = ["United States", "China", "India", "Japan", "Russia", "Saudi Arabia", "South Korea", "Canada", "Brazil", "European Union (27)"]
secondary_y_variables = ["World"]
# Figure out the common start date for a shared x-axis
start_year_consumption = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_consumption = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
start_year_production = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_production = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

oil_start_year = max(start_year_consumption, start_year_production)

start_year = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year = oil_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
title = f"Yearly oil consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Oil consumption (in terawatt-hours)"
secondary_y_axis_title = "World oil consumption (in terawatt-hours)"


custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category
# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

oil_consumption_df["Year"] = pd.to_datetime(oil_consumption_df["Year"], format="%Y")
oil_consumption_df = oil_consumption_df.pivot(index="Year", columns="Entity", values="oil_consumption_twh")
oil_consumption_df = oil_consumption_df[variables + secondary_y_variables]

# See if the overall pricture get better when we restrict the data to only the relative share of the US
oil_consumption_df_usa = oil_consumption_df[["United States", "World"]]
# Calculate the relative share of the US in world oil consumption
oil_consumption_df_usa["United States Share of World"] = oil_consumption_df_usa["United States"] / oil_consumption_df_usa["World"]

fig_oil_consumption = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=oil_consumption_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_01_yearly_oil_consumption_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_oil_consumption.show(renderer=RENDERER)

In [ ]:
variables = ["United States", "Russia", "Saudi Arabia", "Canada", "Iran", "China", "Brazil", "Norway", "European Union (27)"]
secondary_y_variables = ["World"]

oil_production_df = oil_production_df[oil_production_df.Year >= oil_start_year]
start_year = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year = oil_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

title = f"Yearly oil production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Oil production (in terawatt-hours)"
secondary_y_axis_title = "World oil production (in terawatt-hours)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

oil_production_df["Year"] = pd.to_datetime(oil_production_df["Year"], format="%Y")
oil_production_df = oil_production_df.pivot(index="Year", columns="Entity", values="oil_production_twh")
oil_production_df = oil_production_df[variables + secondary_y_variables]


fig_oil_production = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=oil_production_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_01_yearly_oil_production_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_oil_production.show(renderer=RENDERER)

In [ ]:
subplot_titles=(
    f"Yearly oil consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}",
    f"Yearly oil production by country (in terawatt-hours) over the time: {start_year} - {end_year}")
title = f"Yearly oil consumption and production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
secondary_y_variable = "World"

fig_oil_consumption_production_combine = data_graphing_instance.get_combined_production_consumption_graph(
        subplot_titles=list(subplot_titles),
        title="",
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        secondary_y_variable=secondary_y_variable,
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.25,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
        fig_production=fig_oil_production,
        fig_consumption=fig_oil_consumption,
        save_fig=False,
        file_name="chap_01_yearly_oil_consumption_production_combined_graph",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_oil_consumption_production_combine.show(renderer=RENDERER)

#### Gas: Global Production and Consumption over time

In [ ]:
variables = ["United States", "China", "Russia", "Iran", "Canada", "Australia", "Saudi Arabia", "European Union (27)"]
secondary_y_variables = ["World"]
# Figure out the common start date for a shared x-axis
start_year_consumption = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_consumption = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
start_year_production = gas_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year_production = gas_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()
gas_start_year = max(start_year_consumption, start_year_production)

start_year = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.min()
end_year = gas_consumption_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

title = f"Yearly gas consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Gas consumption (in terawatt-hours)"
secondary_y_axis_title = "World gas consumption (in terawatt-hours)"

custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

gas_consumption_df["Year"] = pd.to_datetime(gas_consumption_df["Year"], format="%Y")
gas_consumption_df = gas_consumption_df.pivot(index="Year", columns="Entity", values="gas_consumption_twh")
gas_consumption_df = gas_consumption_df[variables + secondary_y_variables]

fig_gas_consumption = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=gas_consumption_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_01_yearly_gas_consumption_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_consumption.show(renderer=RENDERER)

In [ ]:
variables = ["United States", "China", "Iran", "Canada", "Saudi Arabia", "Mexico", "European Union (27)"]
secondary_y_variables = ["World"]

start_year = gas_start_year
end_year = gas_production_df.query("Entity.isin(@variables) or Entity.isin(@secondary_y_variables)").Year.max()

gas_production_df = gas_production_df[gas_production_df.Year >= gas_start_year]

title = f"Yearly gas production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
y_axis_title = "Production (in terawatt-hours)"
secondary_y_axis_title = "World gas production (in terawatt-hours)"


custom_color_scale = data_graphing_instance.create_custom_diverging_colorscale(
    start_hex="#9b0a7d",
    end_hex="black",
    center_color="grey",
    steps=round((len(variables)+1)/2),
    lightening_factor=0.8,
)
# Extract only the hex color codes from the created list
custom_color_scale_codes = [color[1] for color in custom_color_scale]
world_color = "red"  # Fixed color for the "World" category

# Create a mapping for the colors
color_mapping = {var: custom_color_scale_codes[i % len(custom_color_scale_codes)] for i, var in enumerate(variables)}
color_mapping["World"] = world_color  # Assign the fixed color for "World"

gas_production_df["Year"] = pd.to_datetime(gas_production_df["Year"], format="%Y")
gas_production_df = gas_production_df.pivot(index="Year", columns="Entity", values="gas_production_twh")
gas_production_df = gas_production_df[variables + secondary_y_variables]

fig_gas_production = data_graphing_instance.get_fig_consumption_production_oil_gas(
        data=gas_production_df,
        variables=variables,
        secondary_y_variables=secondary_y_variables,
        title=title,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        secondary_y_axis_title=secondary_y_axis_title,
        color_mapping_dict=color_mapping,
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        save_fig=False,
        file_name="chap_01_yearly_gas_production_by_country",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_production.show(renderer=RENDERER)

In [ ]:
subplot_titles=(f"Yearly gas consumption by country (in terawatt-hours) over the time: {start_year} - {end_year}",
                f"Yearly gas production by country (in terawatt-hours) over the time: {start_year} - {end_year}")
title = f"Yearly gas consumption and production by country (in terawatt-hours) over the time: {start_year} - {end_year}"
x_axis_title = "Time"
secondary_y_variable = "World"

fig_gas_consumption_production_combine = data_graphing_instance.get_combined_production_consumption_graph(
        subplot_titles=subplot_titles,
        title="",
        num_years_interval_x_axis=NUM_YEARS_INTERVAL_X_AXIS,
        x_axis_title=x_axis_title,
        secondary_y_variable=secondary_y_variable,
        rows=2,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.25,
        specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
        fig_production=fig_gas_production,
        fig_consumption=fig_gas_consumption,
        save_fig=False,
        file_name="chap_01_yearly_gas_consumption_production_combined_graph",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_gas_consumption_production_combine.show(renderer=RENDERER)

#### Financial Markets: Oil and Gas OI over time

#### Interest Rate Benchmarks - Absolute Levels

In [17]:
file_name = r"chap_07_interest_rate_comparison_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
interest_rate_comparison_df = pd.read_excel(full_file_path, index_col=0)

data = interest_rate_comparison_df.copy()
variables = interest_rate_comparison_df.columns.tolist()
secondary_yaxis_variables = []
color_discrete_sequence = ["grey", "black", "#9b0a7d"]
title = f"Daily BIS Central Bank Policy Rate and 3M Interbank Rates over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title = "Date"
y_axis_title = "Interest Rate (%)"
secondary_yaxis_title = ""
color_mapping = {
    'USD 3M': "grey",
    'EUR 3M': "black",
    'USD CBPR': "#9b0a7d",
    'EUR CBPR': "lightgrey",
}
fig_interest_rate_comp = data_graphing_instance.get_fig_relationship_main_vars(
        data=data,
        variables=variables,
        secondary_y_variables=secondary_yaxis_variables,
        title="",
        secondary_y_axis_title=secondary_yaxis_title,
        color_discrete_sequence=color_discrete_sequence,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_06_interest_rate_comparison_bis_cbpr_vs_3m_interbank",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_interest_rate_comp.show(renderer="browser")

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/data_graphing/data_grapher.py:1336: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



#### Interest Rate Benchmarks - Relative Levels

In [18]:
file_name = r"chap_07_interest_rate_comparison_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
interest_rate_comparison_df = pd.read_excel(full_file_path, index_col=0)
# Construct the differentials
interest_rate_comparison_df["USD-EUR CBPR"] = interest_rate_comparison_df["USD CBPR"] - interest_rate_comparison_df["EUR CBPR"]
interest_rate_comparison_df["USD-EUR 3M"] = interest_rate_comparison_df["USD 3M"] - interest_rate_comparison_df["EUR 3M"]

variables = ["USD-EUR CBPR", "USD-EUR 3M"]
data = interest_rate_comparison_df.copy()[variables]
secondary_yaxis_variables = []
color_discrete_sequence = ["grey", "black", "#9b0a7d"]
title = f"Daily BIS Central Bank Policy Rate and 3M Interbank Rates differentials (USD-EUR) over the time: {data.index[0].year} - {data.index[-1].year}"
x_axis_title = "Date"
y_axis_title = "Interest Rate differential (%) (USD-EUR)"
secondary_yaxis_title = ""
color_mapping = {
    'USD-EUR CBPR': "black",
    'USD-EUR 3M': "#9b0a7d",
}
fig_interest_rate_diffs_comp = data_graphing_instance.get_fig_relationship_main_vars(
        data=data,
        variables=variables,
        secondary_y_variables=secondary_yaxis_variables,
        title="",
        secondary_y_axis_title=secondary_yaxis_title,
        color_discrete_sequence=color_discrete_sequence,
        x_axis_title=x_axis_title,
        y_axis_title=y_axis_title,
        color_mapping_dict=color_mapping,
        save_fig=False,
        file_name="chap_06_interest_rate_comparison_bis_cbpr_vs_3m_interbank_diffs",
        file_path=FIGURES_PATH,
        width=1200,
        height=800,
        scale=3
        )
# Show the figure
fig_interest_rate_diffs_comp.show(renderer=RENDERER)

/Users/Robert_Hennings/Uni/Master/Seminar/src/seminar_code/data_graphing/data_grapher.py:1336: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



#### Main variables distributions (raw data - normalized)

In [ ]:
file_name = r"chap_07_normed_histogram_data.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
spot_exchange_rate_data_df_normed = pd.read_excel(full_file_path, index_col=0)

start_year = spot_exchange_rate_data_df_normed.index.min().strftime('%Y')
end_year = spot_exchange_rate_data_df_normed.index.max().strftime('%Y')
color_mapping_dict = {
    'EUR/USD': 'grey',
    "WTI Oil": 'black',
    "Nat Gas": 'lightgrey'
}
title=f"Normalized daily EUR/USD spot exchange rate, oil and gas over the time range: {start_year} - {end_year}"
x_axis_title="Daily Observations (Normalized)"
y_axis_title="Probability density"

fig = data_graphing_instance.get_fig_histogram(
    data=spot_exchange_rate_data_df_normed,
    variables=spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    color_discrete_sequence=CAU_COLOR_SCALE,
    color_mapping_dict=color_mapping_dict,
    histnorm="probability density",
    draw_vertical_line_at_0=True,
    showlegend=False,
    save_fig=False,
    file_path=FIGURES_PATH,
    file_name="chap_06_raw_data_normalized_histogram",
    margin_dict=dict(
        l=20,  # Left margin
        r=20,  # Right margin
        t=100,  # Top margin
        b=10   # Bottom margin
    )
)
fig.show(renderer=RENDERER)

#### Main variables distributions (log first differences)

In [ ]:
file_name = r"chap_07_normed_histogram_data_log_first_differences.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
log_diff_spot_exchange_rate_data_df = pd.read_excel(full_file_path, index_col=0)

start_year = log_diff_spot_exchange_rate_data_df.index.min().strftime('%Y')
end_year = log_diff_spot_exchange_rate_data_df.index.max().strftime('%Y')
color_mapping_dict = {
    'EUR/USD': 'grey',
    "WTI Oil": 'black',
    "Nat Gas": 'lightgrey'
}
title=f"Daily log first differences of EUR/USD spot exchange rate, oil and gas over the time range: {start_year} - {end_year}",
x_axis_title="Daily observations (log first differences)"
y_axis_title="Probability density"

fig = data_graphing_instance.get_fig_histogram(
    data=log_diff_spot_exchange_rate_data_df,
    variables=log_diff_spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    color_discrete_sequence=CAU_COLOR_SCALE,
    color_mapping_dict=color_mapping_dict,
    histnorm="probability density",
    draw_vertical_line_at_0=True,
    showlegend=False,
    save_fig=False,
    file_path=FIGURES_PATH,
    file_name="chap_06_log_first_diff_histogram",
    margin_dict=dict(
        l=20,  # Left margin
        r=20,  # Right margin
        t=100,  # Top margin
        b=10   # Bottom margin
    )
)
fig.show(renderer=RENDERER)

#### Tests for Normality (raw data)

In [19]:
file_name = r"norm_test_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
normality_test_results = pd.read_excel(full_file_path, index_col=0)
print(normality_test_results)

                        Test    Statistic       p-value  Significance-level  \
Variable                                                                      
EUR/USD         Shapiro-Wilk     0.988863  1.799963e-22                0.05   
EUR/USD   Kolmogorov-Smirnov     0.799110  0.000000e+00                0.05   
EUR/USD     D'Agostino's K^2    55.280073  9.910253e-13                0.05   
WTI Oil         Shapiro-Wilk     0.981284  1.028844e-28                0.05   
WTI Oil   Kolmogorov-Smirnov     0.999849  0.000000e+00                0.05   
WTI Oil     D'Agostino's K^2   340.084706  1.417570e-74                0.05   
Nat Gas         Shapiro-Wilk     0.863684  4.155066e-60                0.05   
Nat Gas   Kolmogorov-Smirnov     0.937857  0.000000e+00                0.05   
Nat Gas     D'Agostino's K^2  2130.008145  0.000000e+00                0.05   

          p-value < 0.05      Result  
Variable                              
EUR/USD             True  Not-Normal  
EUR/USD      

#### Tests for Normality (log first differences)

In [20]:
file_name = r"norm_test_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
normality_test_results = pd.read_excel(full_file_path, index_col=0)
print(normality_test_results)

                        Test    Statistic       p-value  Significance-level  \
Variable                                                                      
EUR/USD         Shapiro-Wilk     0.978258  1.160894e-30                0.05   
EUR/USD   Kolmogorov-Smirnov     0.490209  0.000000e+00                0.05   
EUR/USD     D'Agostino's K^2   406.729949  4.783148e-89                0.05   
WTI Oil         Shapiro-Wilk     0.869622  2.900955e-59                0.05   
WTI Oil   Kolmogorov-Smirnov     0.463827  0.000000e+00                0.05   
WTI Oil     D'Agostino's K^2  2040.189794  0.000000e+00                0.05   
Nat Gas         Shapiro-Wilk     0.677079  1.879070e-77                0.05   
Nat Gas   Kolmogorov-Smirnov     0.438596  0.000000e+00                0.05   
Nat Gas     D'Agostino's K^2  2698.019653  0.000000e+00                0.05   

          p-value < 0.05      Result  
Variable                              
EUR/USD             True  Not-Normal  
EUR/USD      

#### Tests for Stationarity - ADF Tests (raw data)

In [21]:
file_name = r"adf_test_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
adf_test_df = pd.read_excel(full_file_path, index_col=0)
print(adf_test_df)

                p-value  1% Critical Value  5% Critical Value  \
ADF Statistic                                                   
-1.847164      0.357310          -3.431335          -2.861975   
-1.846637      0.681848          -3.960134          -3.411151   
-2.654927      0.480101          -4.372876          -3.833280   
-0.254286      0.593566          -2.566077          -1.941041   
-2.789238      0.059828          -3.431338          -2.861976   
-2.770357      0.208045          -3.960137          -3.411153   
-3.060291      0.267256          -4.372880          -3.833282   
-0.697102      0.413831          -2.566078          -1.941041   
-4.341106      0.000377          -3.431336          -2.861976   
-4.742770      0.000589          -3.960136          -3.411152   
-4.741953      0.002751          -4.372878          -3.833281   
-1.896565      0.055213          -2.566077          -1.941041   

               10% Critical Value  \
ADF Statistic                       
-1.847164      

#### Tests for Stationarity - ADF Tests (log first differences)

In [22]:
file_name = r"adf_test_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
adf_test_df = pd.read_excel(full_file_path, index_col=0)
print(adf_test_df)

                    p-value  1% Critical Value  5% Critical Value  \
ADF Statistic                                                       
-80.612786     0.000000e+00          -3.431336          -2.861976   
-80.606711     0.000000e+00          -3.960135          -3.411152   
-80.600873     0.000000e+00          -4.372877          -3.833280   
-80.618861     0.000000e+00          -2.566077          -1.941041   
-14.504028     5.868957e-27          -3.431341          -2.861978   
-14.525204     4.186031e-22          -3.960142          -3.411155   
-14.547449     2.075880e-23          -4.372886          -3.833285   
-14.462868     5.907028e-26          -2.566079          -1.941041   
-20.094407     0.000000e+00          -3.431339          -2.861977   
-20.102265     0.000000e+00          -3.960139          -3.411154   
-20.121058     0.000000e+00          -4.372882          -3.833283   
-20.095361     0.000000e+00          -2.566078          -1.941041   

               10% Critical Value

#### Tests for Cointegration (raw data)

In [23]:
file_name = r"cointegration_test_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
cointegration_test_df = pd.read_excel(full_file_path, index_col=0)
print(cointegration_test_df)

                      p-value  Start Time    End Time  Observations Trend  \
Cointegration Score                                                         
-2.966663            0.118344  04-01-1999  01-10-2025          6641     c   
-3.363643            0.133623  04-01-1999  01-10-2025          6641    ct   
-3.635290            0.166672  04-01-1999  01-10-2025          6641   ctt   
-3.267921            0.012603  04-01-1999  01-10-2025          6641     n   
-2.415673            0.317468  04-01-1999  01-10-2025          6641     c   
-2.634044            0.446377  04-01-1999  01-10-2025          6641    ct   
-3.529556            0.204175  04-01-1999  01-10-2025          6641   ctt   
-4.181717            0.000549  04-01-1999  01-10-2025          6641     n   

                    Method  Max Lag Variable X Variable Y  \
Cointegration Score                                         
-2.966663              aeg      NaN    EUR/USD    WTI Oil   
-3.363643              aeg      NaN    EUR/USD

#### Tests for Cointegration (log differences)

In [24]:
file_name = r"cointegration_test_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
cointegration_test_df = pd.read_excel(full_file_path, index_col=0)
print(cointegration_test_df)

                     p-value  Start Time    End Time  Observations Trend  \
Cointegration Score                                                        
-80.626896                 0  05-01-1999  01-10-2025          6638     c   
-80.627022                 0  05-01-1999  01-10-2025          6638    ct   
-80.627661                 0  05-01-1999  01-10-2025          6638   ctt   
-80.626716                 0  05-01-1999  01-10-2025          6638     n   
-80.664269                 0  05-01-1999  01-10-2025          6638     c   
-80.664274                 0  05-01-1999  01-10-2025          6638    ct   
-80.664569                 0  05-01-1999  01-10-2025          6638   ctt   
-80.664270                 0  05-01-1999  01-10-2025          6638     n   

                    Method  Max Lag Variable X Variable Y  \
Cointegration Score                                         
-80.626896             aeg      NaN    EUR/USD    WTI Oil   
-80.627022             aeg      NaN    EUR/USD    WTI Oi

#### Tests for Autocorrelation (raw data)

In [26]:
file_name = r"chap_07_spot_exchange_rate_data_df.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
spot_exchange_rate_data_df = pd.read_excel(full_file_path, index_col=0)

title = f"ACF values for daily observations of the EUR/USD spot exchange rate, oil and gas over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "ACF value"

acf_fig = data_graphing_instance.get_fig_acf(
    data=spot_exchange_rate_data_df,
    variables=spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_acf_plot_raw_series",
    file_path=FIGURES_PATH,
    nlags=30
    )
acf_fig.show(renderer=RENDERER)

#### Tests for Autocorrelation (log first differences)

In [29]:
# Transform to log values and take first difference
spot_exchange_rate_data_df_log_diff = np.log(spot_exchange_rate_data_df).diff().dropna()

title = f"ACF values for daily observations (log first differences) of the spot exchange rate EUR/USD, oil and gas over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "ACF value"

acf_fig = data_graphing_instance.get_fig_acf(
    data=spot_exchange_rate_data_df_log_diff,
    variables=spot_exchange_rate_data_df_log_diff.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_acf_plot_log_diff",
    file_path=FIGURES_PATH,
    nlags=30
    )
acf_fig.show(renderer=RENDERER)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning:

invalid value encountered in log



#### Tests for Partial Autocorrelation (raw data)

In [ ]:
title = f"PACF values for daily observations of the spot exchange rate EUR/USD, oil and gas over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "PACF value"

pacf_fig = data_graphing_instance.get_fig_pacf(
    data=spot_exchange_rate_data_df,
    variables=spot_exchange_rate_data_df.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_pacf_plot_raw_series",
    file_path=FIGURES_PATH,
    nlags=30
    )
pacf_fig.show(renderer=RENDERER)

#### Tests for Partial Autocorrelation (log first differences)

In [ ]:
title = f"PACF values for daily observations (log first differences) of the spot exchange rate EUR/USD, oil and gas over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"
x_axis_title = "Lags"
y_axis_title = "PACF value"

pacf_fig = data_graphing_instance.get_fig_pacf(
    data=spot_exchange_rate_data_df_log_diff,
    variables=spot_exchange_rate_data_df_log_diff.columns.tolist(),
    title="",
    x_axis_title=x_axis_title,
    y_axis_title=y_axis_title,
    save_fig=False,
    file_name="chap_06_pacf_plot_log_diff",
    file_path=FIGURES_PATH,
    nlags=30
    )
pacf_fig.show(renderer=RENDERER)

#### Granger Causality Tests - EUR/USD and oil (raw data)

In [34]:
file_name = r"granger_causality_test_oil_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_oil = pd.read_excel(full_file_path)

variables = granger_test_result_df_oil['Metric'].unique()
secondary_y_variables = ["p-value"]
color_mapping_dict = {
    'p-value': 'red',
    "ssr_ftest": 'darkgrey',
    "ssr_chi2test": 'grey',
    "lrtest": 'black',
    "params_ftest": 'lightgrey',
}
title=f"Granger causality test results testing granger causality of daily observations of oil for EUR/USD over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
secondary_y_axis_title="p-value"

granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_oil,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title=secondary_y_axis_title,
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_oil_raw_series",
    file_path=FIGURES_PATH
    )
granger_causality_test_plot.show(renderer=RENDERER)

#### Granger Causality Tests - EUR/USD and gas (raw data)

In [35]:
file_name = r"granger_causality_test_gas_raw_series.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_gas = pd.read_excel(full_file_path)

variables = granger_test_result_df_gas['Metric'].unique()
secondary_y_variables = ["p-value"]
color_mapping_dict = {
    'p-value': 'red',
    "ssr_ftest": 'darkgrey',
    "ssr_chi2test": 'grey',
    "lrtest": 'black',
    "params_ftest": 'lightgrey',
}
title=f"Granger causality test results testing granger causality of daily observations of oil for EUR/USD over the time: {spot_exchange_rate_data_df.index.min().strftime('%Y')} - {spot_exchange_rate_data_df.index.max().strftime('%Y')}"
secondary_y_axis_title="p-value"

granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_gas,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title=secondary_y_axis_title,
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_oil_raw_series",
    file_path=FIGURES_PATH
    )
granger_causality_test_plot.show(renderer=RENDERER)

#### Granger Causality Tests - EUR/USD and oil (log first differences)

In [36]:
file_name = r"granger_causality_test_oil_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_oil = pd.read_excel(full_file_path)


variables = granger_test_result_df_oil['Metric'].unique()
secondary_y_variables = ["p-value"]
color_mapping_dict = {
    'p-value': 'red',
    "ssr_ftest": 'darkgrey',
    "ssr_chi2test": 'grey',
    "lrtest": 'black',
    "params_ftest": 'lightgrey',
}
title=f"Granger causality test results testing granger causality of daily observations (log first differences) of oil for EUR/USD over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"
secondary_y_axis_title="p-value"

granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_oil,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title=secondary_y_axis_title,
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_oil_log_diff",
    file_path=FIGURES_PATH
    )
granger_causality_test_plot.show(renderer=RENDERER)

#### Granger Causality Tests - EUR/USD and gas (log first differences)

In [ ]:
file_name = r"granger_causality_test_gas_log_diff.xlsx"
full_file_path = rf"{PRESENTATION_DATA}/{file_name}"
granger_test_result_df_gas = pd.read_excel(full_file_path)

variables = granger_test_result_df_gas['Metric'].unique()
secondary_y_variables = ["p-value"]
title=f"Granger causality test results testing granger causality of daily observations (log first differences) of gas for EUR/USD over the time: {spot_exchange_rate_data_df_log_diff.index.min().strftime('%Y')} - {spot_exchange_rate_data_df_log_diff.index.max().strftime('%Y')}"

granger_causality_test_plot = data_graphing_instance.plot_granger_test_results(
    data=granger_test_result_df_gas,
    variables=variables,
    secondary_y_variables=secondary_y_variables,
    color_discrete_sequence=["#9b0a7d", "grey", "black", "darkgrey", "lightgrey"],
    title="",
    secondary_y_axis_title="p-value",
    x_axis_title="Lag",
    y_axis_title="Test-Statistic",
    color_mapping_dict=color_mapping_dict,
    significance_level=0.05,
    margin_dict=dict(
            l=20,  # Left margin
            r=20,  # Right margin
            t=50,  # Top margin
            b=10   # Bottom margin
            ),
    showlegend=False,
    save_fig=False,
    file_name="chap_06_granger_causality_test_gas_log_diff",
    file_path=FIGURES_PATH
    )
granger_causality_test_plot.show(renderer=RENDERER)